In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pyautogui
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from datetime import datetime

In [17]:
def ler_tabelas(url, find_button=False):
    # INICIALIZAR O DRIVER DO NAVEGADOR
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run in headless mode
    driver = webdriver.Chrome()

    #FAZER O DRIVER IR PARA A URL ESPECIFICADA
    driver.get(url)

    #esperar 10 segundos para a pagina carregar
    time.sleep(5)

    if find_button:
        # Wait for the element with ID "fechar" to appear
        wait = WebDriverWait(driver, 40)
        fechar_element = wait.until(EC.presence_of_element_located((By.ID, 'fechar')))

        date_min = driver.find_element(By.ID, "dateMin")
        date_min.send_keys("2020")

        date_max = driver.find_element(By.ID, "dateMax")
        date_max.send_keys("2023")
        # Find the button by ID and click on it
        button_xpath = '//*[@id="see_all_quotes_history"]'
        # Wait for the button to be clickable (adjust timeout as needed)
        button = driver.find_element(By.XPATH, button_xpath)

        # Click on the button
        button.click()
        time.sleep(5)

    #obter o HTML da pagina
    html = driver.page_source

    #Fechar driver do chrome
    driver.quit()

    #Usar o BS4 para analisar o meu html da url
    soup = BeautifulSoup(html, 'html.parser')

    #Encontrar todas as tabelas da pagina
    tables = soup.find_all('table')

    #converter cada tabela em um Dataframe do PANDAS e analisar os Dataframes em uma lista
    dfs = [pd.read_html(str(table))[0] for table in tables]

    return dfs

In [18]:
url = "https://www.infomoney.com.br/cotacoes/empresas-b3/"
tabelas = ler_tabelas(url)

In [19]:
def click_on_element(driver, method, path, wait_time=30):
    try:
        WebDriverWait(driver, wait_time).until(EC.element_to_be_clickable((method, path))).click()
        return True
    except:
        print(f'Can not click on {path}')
        return False

In [20]:
def enter_text(driver, method, path, text):
    try:
        element_cnpj = driver.find_element(by=method, path=path)
        element_cnpj.send_keys(text)
    except:
        print('Could not enter text')

In [21]:
def get_hist_values(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run in headless mode
    driver = webdriver.Chrome()

    #FAZER O DRIVER IR PARA A URL ESPECIFICADA
    driver.get(url)
    # driver.maximize_window()

    #esperar 10 segundos para a pagina carregar
    time.sleep(random.uniform(2, 10))

    print('Switching to iFrame')
    driver.switch_to.frame(driver.find_element(By.TAG_NAME, "iframe"))

    click_on_element(driver=driver, method=By.XPATH, path='//*[@id="fechar"]')
    print('Click on close button')

    print('Switching to the main frame')
    driver.switch_to.default_content()
    
    # input_min = driver.find_element(by=By.XPATH, value='//*[@id="dateMin"]')
    click_on_element(driver=driver, method=By.XPATH, path='//*[@id="dateMin"]')
    # input_min.send_keys("01/01/2023")
    pyautogui.hotkey('0', '1', '/', '0', '1', '/','2', '0','1', '5',interval=0.3)

    today = datetime.now().strftime("%d/%m/%Y").strip()
    click_on_element(driver=driver, method=By.XPATH, path='//*[@id="dateMax"]')
    pyautogui.hotkey(today[0], today[1], today[2], today[3], today[4], today[5], today[6], today[7], today[8], today[9],interval=0.3)

    click_on_element(driver=driver, method=By.XPATH, path='//*[@id="see_all_quotes_history"]')
    print('Clicked on see data')

    # EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="quotes_history"]/tbody'))
    try:
        WebDriverWait(driver, 60).until(EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="quotes_history"]/tbody')))
    except:
        print('Loading not completed')


    time.sleep(20)
    #obter o HTML da pagina
    html = driver.page_source

    #Fechar driver do chrome
    driver.quit()

    #Usar o BS4 para analisar o meu html da url
    soup = BeautifulSoup(html, 'html.parser')

    #Encontrar todas as tabelas da pagina
    tables = soup.find_all('table')

    #converter cada tabela em um Dataframe do PANDAS e analisar os Dataframes em uma lista
    dfs = [pd.read_html(str(table))[0] for table in tables]

    return dfs    

In [22]:
url_historico = "https://www.infomoney.com.br/cotacoes/b3/acao/magazine-luiza-mglu3/historico/"
tabelas_historico = get_hist_values(url_historico)

Switching to iFrame
Click on close button
Switching to the main frame
Clicked on see data


In [27]:
#Enquanto o carregamento não funcionar, continuar tentando...colocar isso para três tentativas
while tabelas_historico[0]['DATA'].iloc[0] == 'Carregando...':
    tabelas_historico = get_hist_values(url_historico)

In [28]:
if tabelas_historico:
    #obter o primeiro dataframe da lista de tabelas
    df = tabelas_historico[0]
    datas = df['DATA'].tolist()
    df['FECHAMENTO'] = df['FECHAMENTO'].astype(float)/100.0 

    #Converter a coluna de data para o tipo datetime
    df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y')

    # Defina a faixa desejada
    data_inicio = '2020-01-01'
    data_fim = '2024-10-01'
    df_filtrado = df[(df['DATA'] >= data_inicio) & (df['DATA'] < data_fim)]

    print(df_filtrado[['DATA','FECHAMENTO']])

else:
    #informar erro ou que não foi encontraada nenhuma tabela
    print("Não foi encontrada nenhuma tabela ou coluna")

          DATA  FECHAMENTO
0   2023-11-13      0.0173
1   2023-11-10      0.0180
2   2023-11-09      0.0171
3   2023-11-08      0.0178
4   2023-11-07      0.0177
..         ...         ...
495 2021-11-19      0.0927
496 2021-11-18      0.0899
497 2021-11-17      0.0927
498 2021-11-16      0.0974
499 2021-11-12      0.1115

[500 rows x 2 columns]


In [29]:
df.tail()

,DATA,ABERTURA,FECHAMENTO,VARIAÇÃO,MÍNIMO,MÁXIMO,VOLUME
495,2021-11-19,905,0.0927,311,894,987,"887,06M"
496,2021-11-18,936,0.0899,-302,899,960,"708,63M"
497,2021-11-17,991,0.0927,-483,919,1014,"1,25B"
498,2021-11-16,1119,0.0974,-1265,974,1140,"1,46B"
499,2021-11-12,1265,0.1115,-1832,1115,1278,"1,68B"


In [30]:
df.head()

,DATA,ABERTURA,FECHAMENTO,VARIAÇÃO,MÍNIMO,MÁXIMO,VOLUME
0,2023-11-13,178,0.0173,-389,162,188,"396,07M"
1,2023-11-10,173,0.0180,526,173,181,"187,47M"
2,2023-11-09,183,0.0171,-393,168,183,"304,17M"
3,2023-11-08,175,0.0178,57,172,202,"654,62M"
4,2023-11-07,142,0.0177,2378,141,178,"490,16M"
